import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import time
start = time.time()

import data

In [2]:
raw_path = "raw/"

df_sample_submission = pd.read_csv(raw_path + "sample_submission.csv")
df_test_logs = pd.read_csv(raw_path + "test_logs.csv")
df_train_logs = pd.read_csv(raw_path + "train_logs.csv")
df_train_scores = pd.read_csv(raw_path + "train_scores.csv")

overview of datasets

In [3]:
print(df_sample_submission.shape)
df_sample_submission.head()

(3, 2)


,id,score
0,0000aaaa,1.0
1,2222bbbb,2.0
2,4444cccc,3.0


In [4]:
print(df_test_logs.shape)
df_test_logs.head(10)

(6, 11)


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0
5,4444cccc,2,184996,185052,56,Input,q,q,q,1,1


In [5]:
print(df_train_logs.shape)
df_train_logs.head(10)

(8405898, 11)


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1
6,001519c8,7,107469,107596,127,Input,q,q,q,4,1
7,001519c8,8,107659,107766,107,Input,q,q,q,5,1
8,001519c8,9,107743,107852,109,Input,q,q,q,6,1
9,001519c8,10,107840,107978,138,Input,Space,Space,,7,1


In [6]:
print(df_train_scores.shape)
df_train_scores.head()

(2471, 2)


,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


combine train logs with train scores

In [7]:
df_train = pd.merge(df_train_logs, df_train_scores, on='id')
print(df_train.shape)
df_train.head()

(8405898, 12)


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0,3.5
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1,3.5
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1,3.5


looking at one users log data to demonstrate the data processing work before scaling to all users

In [8]:
df_tmp = df_train.loc[df_train['id'] == "0042269b", :]

print(df_tmp.shape)
df_tmp.head(10)

(4136, 12)


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
5011,0042269b,1,4441,4540,99,Nonproduction,Leftclick,Leftclick,NoChange,0,0,6.0
5012,0042269b,2,21276,21276,0,Nonproduction,Shift,Shift,NoChange,0,0,6.0
5013,0042269b,3,21335,21430,95,Input,q,q,q,1,1,6.0
5014,0042269b,4,21533,21665,132,Input,q,q,q,2,1,6.0
5015,0042269b,5,21594,21742,148,Input,q,q,q,3,1,6.0
5016,0042269b,6,21692,21832,140,Input,q,q,q,4,1,6.0
5017,0042269b,7,21743,21871,128,Input,q,q,q,5,1,6.0
5018,0042269b,8,21852,21973,121,Input,q,q,q,6,1,6.0
5019,0042269b,9,21934,22037,103,Input,q,q,q,7,1,6.0
5020,0042269b,10,22030,22092,62,Input,Space,Space,,8,1,6.0


What is to be done with each feature?

- id — save as user id reference
- event_id — can be removed
  - Useful for ordering events chronologically in processing, but not needed directly for final user data
- down_time, up_time, action_time
  - min, max, median, mean, std
- activity
  - Events include: Input, Remove/Cut, Nonproduction, Replace, Paste, MoveFrom
  - Map any "Move From" actions to one MoveFrom event
- down_event
  - map to 4 categories -> alphanumeric, special_characters, control_keys, unknown
  - proportion of each (not count because may have undue influence)
- up_event -> can be removed (essentially the same as down_event(?))
- text_change
  - map to type of change -> many, single, not (with origin to target)
  - split feature into origin_text, and target_text
  - Possibly add feature to indicate inc/dec (but maybe not necessary)
- cursor_position
  - "linear deviation" -> error metric (e.g., RMSE, MAE, etc) between actual positions and positions if text was simply streamed
- word_count
  - min, max, median, mean, std

aggregate stats (min, max, median, mean, std) from the following features:
- down_time
- up_time
- action_time
- word_count

In [9]:
agg_functions = {
    'down_time': ['min', 'max', 'median', 'mean', 'std'], 
    'up_time': ['min', 'max', 'median', 'mean', 'std'], 
    'action_time': ['min', 'max', 'median', 'mean', 'std'], 
    'word_count': ['min', 'max', 'median', 'mean', 'std']
}
df_tmp_agg = df_tmp.groupby('id').agg(agg_functions)

df_tmp_agg.reset_index(inplace=True)
df_tmp_agg.columns = ["id"] + ['_'.join(col).strip() for col in df_tmp_agg.columns[1:].values]

df_tmp_agg

,id,down_time_min,down_time_max,down_time_median,down_time_mean,down_time_std,up_time_min,up_time_max,up_time_median,up_time_mean,...,action_time_min,action_time_max,action_time_median,action_time_mean,action_time_std,word_count_min,word_count_max,word_count_median,word_count_mean,word_count_std
0,0042269b,4441,1771219,759582.0,828491.775145,489500.796565,4540,1771669,759674.5,828593.612911,...,0,3005,94.0,101.837766,82.383766,0,404,193.0,194.772727,108.935068


In [10]:
df_tmp.columns

Index(['id', 'event_id', 'down_time', 'up_time', 'action_time', 'activity',
       'down_event', 'up_event', 'text_change', 'cursor_position',
       'word_count', 'score'],
      dtype='object')

In [11]:
df_tmp[['activity', 'down_event', 'text_change', 'cursor_position', 'word_count', 'score']].head()

,activity,down_event,text_change,cursor_position,word_count,score
5011,Nonproduction,Leftclick,NoChange,0,0,6.0
5012,Nonproduction,Shift,NoChange,0,0,6.0
5013,Input,q,q,1,1,6.0
5014,Input,q,q,2,1,6.0
5015,Input,q,q,3,1,6.0


inspecting activity data shows 6 types of activities

In [12]:
df_train.activity.value_counts()

Input                                     6726796
Remove/Cut                                 970158
Nonproduction                              703851
Replace                                      4448
Paste                                         599
Move From [1306, 1371] To [1061, 1126]          2
Move From [13, 65] To [9, 61]                   1
Move From [274, 314] To [299, 339]              1
Move From [624, 625] To [845, 846]              1
Move From [1861, 2063] To [1766, 1968]          1
Move From [1766, 1968] To [1861, 2063]          1
Move From [2091, 2179] To [252, 340]            1
Move From [923, 1077] To [340, 494]             1
Move From [0, 1] To [590, 591]                  1
Move From [999, 1000] To [1000, 1001]           1
Move From [0, 75] To [1, 76]                    1
Move From [1651, 1769] To [1565, 1683]          1
Move From [61, 136] To [0, 75]                  1
Move From [623, 632] To [624, 633]              1
Move From [75, 134] To [304, 363]               1


In [13]:
df_train.activity.apply(lambda x: "MoveFrom" if x[:9].lower() == "move from" else x).value_counts()

Input            6726796
Remove/Cut        970158
Nonproduction     703851
Replace             4448
Paste                599
MoveFrom              46
Name: activity, dtype: int64

inspecting down_event data (theoretically the same as up_event data) shows 4 general categories:
- alphanumeric
- special characters
- control keys (e.g., Space, Backspace, etc.)
- unknown

In [14]:
[[event, count] for event, count in zip(df_train.down_event.value_counts().index, df_train.down_event.value_counts().values)]

[['q', 5365319],
 ['Space', 1207128],
 ['Backspace', 964089],
 ['Shift', 334227],
 ['ArrowRight', 121338],
 ['Leftclick', 91126],
 ['ArrowLeft', 63217],
 ['.', 59586],
 [',', 46806],
 ['ArrowDown', 30874],
 ['ArrowUp', 28189],
 ['Enter', 22689],
 ['CapsLock', 12568],
 ["'", 11170],
 ['Delete', 10965],
 ['Unidentified', 6151],
 ['Control', 4885],
 ['"', 4102],
 ['-', 3843],
 ['?', 3155],
 [';', 1785],
 ['=', 1155],
 ['Tab', 1081],
 ['/', 874],
 ['Rightclick', 655],
 [':', 634],
 ['(', 611],
 [')', 578],
 ['\\', 554],
 ['ContextMenu', 552],
 ['End', 547],
 ['!', 546],
 ['Meta', 479],
 ['Alt', 466],
 ['[', 369],
 ['c', 359],
 ['v', 311],
 ['NumLock', 273],
 ['Insert', 226],
 ['Home', 221],
 ['z', 207],
 ['AudioVolumeDown', 155],
 ['F2', 146],
 ['a', 140],
 ['x', 127],
 ['AudioVolumeUp', 105],
 ['$', 97],
 ['>', 89],
 [']', 81],
 ['*', 69],
 ['%', 65],
 ['&', 60],
 ['Dead', 53],
 ['s', 44],
 ['Escape', 41],
 ['ModeChange', 37],
 ['F3', 36],
 ['<', 34],
 ['AudioVolumeMute', 31],
 ['F15', 30

Here are the defined groups of down_events based on train data

In [15]:
known_events = {
    "alphanumeric": ["q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", "h", "j", "k", "l", "z", "x", "c", "v", "b", "n", "m", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0"],
    "special_character": [",", ".", "/", ";", "'", "[", "]", "\\", "`", "-", "=", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "+", "{", "}", "|", ":", '"', "<", ">", "?", "~"],
    "control_keys": ["Space", "Backspace", "Shift", "ArrowRight", "Leftclick", "ArrowLeft", "ArrowDown", "ArrowUp", "Enter", "CapsLock", "Delete", "Unidentified", "Control", "Tab", "Rightclick", "ContextMenu", "End", "Meta", "Alt", "NumLock", "Insert", "Home", "AudioVolumeDown", "F2", "AudioVolumeUp", "Dead", "Escape", "ModeChange", "F3", "AudioVolumeMute", "F15", "ScrollLock", "Process", "PageDown", "PageUp", "Middleclick", "MediaTrackNext", "MediaPlayPause", "Clear", "AltGraph", "MediaTrackPrevious", "Pause", "Cancel", "F1", "F10", "F6", "F4", "F5", "F7", "F8", "F9", "F12", "F11", "Unknownclick", "F13", "F14"],
}


In [16]:
def get_event_type(event):
    if event.lower() in known_events["alphanumeric"]:
        return "alphanumeric"
    elif event.lower() in known_events["special_character"]:
        return "special_character"
    elif event.lower() in [e.lower() for e in known_events["control_keys"]]:
        return "control_keys"
    else:
        return "unknown"
    
df_train.down_event.apply(lambda x: get_event_type(x)).value_counts()

alphanumeric         5366705
control_keys         2902756
special_character     136392
unknown                   45
Name: down_event, dtype: int64

inspection of text_change data shows many variety of types of changes

In [17]:
text_change_data = [[event, count] for event, count in zip(df_train.text_change.value_counts().index, df_train.text_change.value_counts().values)]
text_change_data

[['q', 6128208],
 [' ', 1376829],
 ['NoChange', 703851],
 ['.', 67797],
 [',', 54797],
 ['\n', 27977],
 ["'", 13192],
 ['"', 4659],
 ['-', 4492],
 ['?', 3507],
 [';', 2905],
 ['=', 2288],
 ['/', 1354],
 ['\\', 1100],
 [':', 813],
 ['(', 733],
 ['[', 725],
 [')', 660],
 ['!', 639],
 ['q => q', 238],
 ['qqqq => q', 205],
 ['qqq => q', 191],
 ['>', 177],
 ['qqqqqq => q', 165],
 [']', 157],
 ['qq => q', 155],
 ['qqqqq => q', 142],
 ['. ', 137],
 ['qqq  => q', 130],
 ['qqqq  => q', 120],
 ['qqqqqqq => q', 114],
 ['$', 112],
 ['*', 108],
 ['qqq ', 90],
 ['qqqqq  => q', 88],
 ['qqq', 83],
 ['qqqq ', 81],
 ['qqqq', 78],
 ['&', 77],
 ['%', 77],
 ['qqqqqqqq => q', 76],
 ['qqqqqqqqq => q', 75],
 ['qqqqqq  => q', 74],
 ['qq  => q', 70],
 ['<', 67],
 ['qqqqq', 61],
 ['qqqqqqq  => q', 60],
 ['qqqqqq', 58],
 ['qq', 55],
 ['qqqqq ', 51],
 ['qq ', 49],
 ['qqqqqqqqqq => q', 41],
 ['qqqqqq ', 41],
 ['qqqqqqqq  => q', 41],
 ['\n ', 40],
 ['+', 40],
 ['qqqqqqqqqqq => q', 38],
 ['qqqqqqqqq  => q', 37],
 ['q

inspecting text changes that involve `" => "` changes

In [18]:
import re
def is_only_qs(s):
    return bool(re.match(r'^q+$', s))

def remove_special_characters(s):
    return re.sub(r'[^a-zA-Z0-9]', '', s.strip().replace(" ", ""))

def map_q(s):
    if len(s) > 1 and is_only_qs(s):
        s = "many_q"
    elif len(s) == 1 and s == "q":
        s = "single_q"
    elif s == "":
        s = "not_q"
    return s

replace_changes = set()
for event, count in text_change_data:
    if "=>" in event:
        parts = event.split("=>")
        left = remove_special_characters(parts[0])
        left = map_q(left)
        right = remove_special_characters(parts[1])
        right = map_q(right)
        change = left + " => " + right
        replace_changes.add(change)
        
replace_changes

{'many_q => many_q',
 'many_q => not_q',
 'many_q => single_q',
 'not_q => many_q',
 'not_q => not_q',
 'not_q => single_q',
 'single_q => many_q',
 'single_q => not_q',
 'single_q => single_q'}

inspecting other kinds of text changes

In [19]:
other_changes = set()
for event, count in text_change_data:
    if "=>" not in event:
        change = remove_special_characters(event)
        change = map_q(change)
        other_changes.add(change)
        
other_changes

{'NoChange', 'many_q', 'not_q', 'single_q'}

demo of "linear deviation" for cursor positions
- Assumption: more frequent or major edits will result in greater deviation
- Two lists of positions
  - Actual cursor positions
  - Stream cursor positions (no edits)

For "stream" list, iteratively increase cursor_position by 1 unless the text_change value is "NoChange".

In [20]:
df_tmp[["text_change", "cursor_position"]]

,text_change,cursor_position
5011,NoChange,0
5012,NoChange,0
5013,q,1
5014,q,2
5015,q,3
...,...,...
9142,q,488
9143,q,489
9144,q,490
9145,.,491


In [21]:
stream_cursor_position = []
pos = 0
for index, row in df_tmp.iterrows():
    if row["text_change"] != "NoChange":
        pos += 1
    stream_cursor_position.append(pos)
    
df_tmp["stream_cursor_position"] = stream_cursor_position
df_tmp[["text_change", "cursor_position", "stream_cursor_position"]]

,text_change,cursor_position,stream_cursor_position
5011,NoChange,0,0
5012,NoChange,0,0
5013,q,1,1
5014,q,2,2
5015,q,3,3
...,...,...,...
9142,q,488,3958
9143,q,489,3959
9144,q,490,3960
9145,.,491,3961


In [22]:
np.mean(np.abs(df_tmp.cursor_position - df_tmp.stream_cursor_position))

1238.7553191489362

Putting everything together...

1. Create cleaned data
2. Transform user log subsets into 1 row

In [23]:
# known down events
known_events = {
    "alphanumeric": ["q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", "h", "j", "k", "l", "z", "x", "c", "v", "b", "n", "m", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0"],
    "special_character": [",", ".", "/", ";", "'", "[", "]", "\\", "`", "-", "=", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "+", "{", "}", "|", ":", '"', "<", ">", "?", "~"],
    "control_keys": ["Space", "Backspace", "Shift", "ArrowRight", "Leftclick", "ArrowLeft", "ArrowDown", "ArrowUp", "Enter", "CapsLock", "Delete", "Unidentified", "Control", "Tab", "Rightclick", "ContextMenu", "End", "Meta", "Alt", "NumLock", "Insert", "Home", "AudioVolumeDown", "F2", "AudioVolumeUp", "Dead", "Escape", "ModeChange", "F3", "AudioVolumeMute", "F15", "ScrollLock", "Process", "PageDown", "PageUp", "Middleclick", "MediaTrackNext", "MediaPlayPause", "Clear", "AltGraph", "MediaTrackPrevious", "Pause", "Cancel", "F1", "F10", "F6", "F4", "F5", "F7", "F8", "F9", "F12", "F11", "Unknownclick", "F13", "F14"],
}

# map down events to types of events
def get_event_type(event):
    if event.lower() in known_events["alphanumeric"]:
        return "alphanumeric"
    elif event.lower() in known_events["special_character"]:
        return "special_character"
    elif event.lower() in [e.lower() for e in known_events["control_keys"]]:
        return "control_keys"
    else:
        return "unknown"

# check if a string is only q's
def is_only_qs(s):
    return bool(re.match(r'^q+$', s))

# remove special characters from a string
def remove_special_characters(s):
    return re.sub(r'[^a-zA-Z0-9]', '', s.strip().replace(" ", ""))

# map q's to types of q's (single q, many q's, not q)
def map_q(s):
    if len(s) > 1 and is_only_qs(s):
        s = "many_q"
    elif len(s) == 1 and s == "q":
        s = "single_q"
    elif s == "":
        s = "not_q"
    return s

# map text change data to types of changes
def clean_text_change(text):
    text = text.lower()
    if "=>" in text:
        parts = text.split("=>")
        left = remove_special_characters(parts[0])
        left = map_q(left)
        right = remove_special_characters(parts[1])
        right = map_q(right)
        change = left + " => " + right
        return change
    else:
        change = remove_special_characters(text)
        change = map_q(change)
        return change

In [24]:
df_train_clean = df_train.copy()

def clean_df(df):
    # drop up_event columns
    df.drop(columns=["up_event"], inplace=True)

    # clean activity column
    df.activity = df.activity.apply(lambda x: "movefrom" if x[:9].lower() == "move from" else x.lower())

    # clean down_event column
    df.down_event = df.down_event.apply(lambda x: get_event_type(x).lower())

    # clean text_change column
    df.text_change = df.text_change.apply(lambda x: clean_text_change(x).lower())

    return df

df_train_clean = clean_df(df_train_clean)
df_train_clean.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,text_change,cursor_position,word_count,score
0,001519c8,1,4526,4557,31,nonproduction,control_keys,nochange,0,0,3.5
1,001519c8,2,4558,4962,404,nonproduction,control_keys,nochange,0,0,3.5
2,001519c8,3,106571,106571,0,nonproduction,control_keys,nochange,0,0,3.5
3,001519c8,4,106686,106777,91,input,alphanumeric,single_q,1,1,3.5
4,001519c8,5,107196,107323,127,input,alphanumeric,single_q,2,1,3.5


confirm features are cleaned

In [25]:
df_train_clean.activity.value_counts()

input            6726796
remove/cut        970158
nonproduction     703851
replace             4448
paste                599
movefrom              46
Name: activity, dtype: int64

In [26]:
df_train_clean.down_event.value_counts()

alphanumeric         5366705
control_keys         2902756
special_character     136392
unknown                   45
Name: down_event, dtype: int64

In [27]:
df_train_clean.text_change.value_counts()

single_q                6128270
not_q                   1565711
nochange                 703851
many_q => single_q         3633
many_q                     3618
many_q => not_q             345
single_q => single_q        264
many_q => many_q             67
not_q => not_q               64
not_q => single_q            36
single_q => not_q            32
not_q => many_q               4
single_q => many_q            3
Name: text_change, dtype: int64

converting each users logs to one row for final processed data (deomnstrate first on one user)

In [28]:
df_train_clean.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,text_change,cursor_position,word_count,score
0,001519c8,1,4526,4557,31,nonproduction,control_keys,nochange,0,0,3.5
1,001519c8,2,4558,4962,404,nonproduction,control_keys,nochange,0,0,3.5
2,001519c8,3,106571,106571,0,nonproduction,control_keys,nochange,0,0,3.5
3,001519c8,4,106686,106777,91,input,alphanumeric,single_q,1,1,3.5
4,001519c8,5,107196,107323,127,input,alphanumeric,single_q,2,1,3.5


In [29]:
known_activities = [item.lower() for item in df_train_clean.activity.value_counts().index]
known_down_events = [item.lower() for item in df_train_clean.down_event.value_counts().index]
known_text_changes = [item.lower() for item in df_train_clean.text_change.value_counts().index] + ["unknown"]

In [30]:
def get_proportion_data(df, column, known_items):
    df_tmp = df.copy()
    df_tmp = df_tmp[column].value_counts(normalize=True).to_frame().T.reset_index(drop=True)
    df_tmp.columns = [column + '_' + col.lower() for col in df_tmp.columns]
    for item in known_items:
        if column + '_' + item not in df_tmp.columns:
            df_tmp[column + '_' + item] = 0
    df_tmp = df_tmp[sorted([column + '_' + item for item in known_items])]
    return df_tmp

In [31]:
def summarize_user_logs(df, user_id):
    df_user = df.loc[df["id"] == user_id, :]
    agg_functions = {
        'down_time': ['min', 'max', 'median', 'mean', 'std'], 
        'up_time': ['min', 'max', 'median', 'mean', 'std'], 
        'action_time': ['min', 'max', 'median', 'mean', 'std'], 
        'word_count': ['min', 'max', 'median', 'mean', 'std']
    }
    df_user_agg = df_user.groupby('id').agg(agg_functions)
    df_user_agg.reset_index(inplace=True)
    df_user_agg.columns = ["id"] + ['_'.join(col).strip().lower() for col in df_user_agg.columns[1:].values]

    # get type proportions of each categorical feature
    df_user_activity_proportions = get_proportion_data(df_user, "activity", known_activities)
    df_user_down_event_proportions = get_proportion_data(df_user, "down_event", known_down_events)
    df_user_text_change_proportions = get_proportion_data(df_user, "text_change", known_text_changes)

    # add proportion dataframes to df_user_agg as new columns
    df_user_agg = pd.concat([df_user_agg, df_user_activity_proportions, df_user_down_event_proportions, df_user_text_change_proportions], axis=1)

    # sort df_user by event id
    df_user.sort_values(by=["event_id"], inplace=True)
    # compute "stream" cursor position
    stream_cursor_position = []
    pos = 0
    for index, row in df_user.iterrows():
        if row["text_change"] != "nochange":
            pos += 1
        stream_cursor_position.append(pos)
    # add stream cursor position to df_user
    df_user["stream_cursor_position"] = stream_cursor_position
    # compute mean absolute error between cursor position and stream cursor position
    mae_cursor_position = np.mean(np.abs(df_user.cursor_position - df_user.stream_cursor_position))
    # add mae to df_user_agg
    df_user_agg["mae_cursor_position"] = mae_cursor_position

    # add score to df_user_agg
    df_user_agg["score"] = df_user["score"].values[0]

    # normalize column names
    # replace "=>" with "to" in text_change column
    df_user_agg.columns = [col.replace("=>", "to") for col in df_user_agg.columns]
    # replace " " with "_" in column names
    df_user_agg.columns = [col.replace(" ", "_") for col in df_user_agg.columns]
    # replace "-" with "_" in column names
    df_user_agg.columns = [col.replace("-", "_") for col in df_user_agg.columns]
    # replace any other special characters with "" in column names (except "_")
    df_user_agg.columns = [re.sub(r'[^a-zA-Z0-9_]', '', col) for col in df_user_agg.columns]

    # sort columns
    columns_id = ["id"]
    columns_down_time = [col for col in df_user_agg.columns if col.startswith("down_time")]
    columns_up_time = [col for col in df_user_agg.columns if col.startswith("up_time")]
    columns_action_time = [col for col in df_user_agg.columns if col.startswith("action_time")]
    columns_activity = [col for col in df_user_agg.columns if col.startswith("activity")]
    columns_down_event = [col for col in df_user_agg.columns if col.startswith("down_event")]
    columns_text_change = [col for col in df_user_agg.columns if col.startswith("text_change")]
    columns_cursor_position = ["mae_cursor_position"]
    columns_word_count = [col for col in df_user_agg.columns if col.startswith("word_count")]
    columns_score = ["score"]

    columns_sort = columns_id + columns_score + columns_cursor_position + \
        columns_down_time + columns_up_time + columns_action_time + columns_word_count + \
        columns_activity + columns_down_event + columns_text_change
    df_user_agg = df_user_agg[columns_sort]

    return df_user_agg

summarize_user_logs(df_train_clean, "0042269b").T

,0
id,0042269b
score,6.0
mae_cursor_position,1238.755319
down_time_min,4441
down_time_max,1771219
down_time_median,759582.0
down_time_mean,828491.775145
down_time_std,489500.796565
up_time_min,4540
up_time_max,1771669


Now scaling it to all users...

In [32]:
user_ids = df_train_clean.id.unique()
df_user_summaries = pd.DataFrame()

for user_id in user_ids:
    df_user = summarize_user_logs(df_train_clean, user_id)
    df_user_summaries = pd.concat([df_user_summaries, df_user], axis=0)

In [33]:
df_user_summaries.reset_index(drop=True, inplace=True)
df_user_summaries

,id,score,mae_cursor_position,down_time_min,down_time_max,down_time_median,down_time_mean,down_time_std,up_time_min,up_time_max,...,text_change_nochange,text_change_not_q,text_change_not_q_to_many_q,text_change_not_q_to_not_q,text_change_not_q_to_single_q,text_change_single_q,text_change_single_q_to_many_q,text_change_single_q_to_not_q,text_change_single_q_to_single_q,text_change_unknown
0,001519c8,3.5,527.046930,4526,1801877,891716.0,8.481808e+05,395112.665961,4557,1801969,...,0.046930,0.190849,0.0,0.0,0.0,0.758702,0.0,0.0,0.000000,0
1,0022f953,3.5,380.774654,30623,1788842,407673.0,5.188553e+05,384959.404177,30853,1788969,...,0.103504,0.204156,0.0,0.0,0.0,0.691932,0.0,0.0,0.000000,0
2,0042269b,6.0,1238.755319,4441,1771219,759582.0,8.284918e+05,489500.796565,4540,1771669,...,0.042311,0.167795,0.0,0.0,0.0,0.787476,0.0,0.0,0.000000,0
3,0059420b,2.0,152.393316,41395,1404394,848240.5,7.854830e+05,385205.014399,41513,1404469,...,0.063625,0.198586,0.0,0.0,0.0,0.736504,0.0,0.0,0.000000,0
4,0075873a,4.0,640.161596,78470,1662390,686588.0,7.133542e+05,405576.409034,78693,1662472,...,0.028447,0.195575,0.0,0.0,0.0,0.775978,0.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,3.5,1474.277696,22467,1791581,735095.0,7.361019e+05,503882.020411,22551,1791649,...,0.039882,0.198776,0.0,0.0,0.0,0.760709,0.0,0.0,0.000000,0
2467,ffbef7e5,4.0,206.658986,21732,1799124,748404.5,8.419662e+05,512744.745940,22027,1799174,...,0.056836,0.205453,0.0,0.0,0.0,0.737327,0.0,0.0,0.000384,0
2468,ffccd6fd,1.5,133.272935,23482,1959273,1506525.0,1.229015e+06,514320.848199,23570,1959363,...,0.041136,0.622266,0.0,0.0,0.0,0.336598,0.0,0.0,0.000000,0
2469,ffec5b38,5.0,398.298890,19885,1508335,573912.0,5.765185e+05,334477.976640,20088,1508504,...,0.021900,0.177668,0.0,0.0,0.0,0.799815,0.0,0.0,0.000000,0


In [34]:
# save df_user_summaries to csv (full)
data_path = "data/"
df_user_summaries.to_csv(data_path + "df_user_summaries.csv", index=False)

In [35]:
# a possible suggested subset of features
# - id - keep
# - score - keep
# - mae_cursor_position - keep
# - down_time -> max, mean, std - keep
# - up_time -> max, mean, std - keep (maybe drop since highly correlated with down_time)
# - action_time -> mean, std - keep
# - word_count -> max, mean, std - keep
# - activity -> keep all, eliminate via feature selection
# - down_event -> keep all, eliminate via feature selection
# - text_change -> keep all, eliminate via feature selection (refer to next 2 code blocks for reference)
#    - could pool features together (e.g. "x_to_y" -> "replacements", "many_q" -> "move/cut", "nochange" vs "change", etc.)

# pooling text_change features that involve "replace" activity
df_user_summaries["text_change_replacements"] = df_user_summaries[[
    'text_change_many_q_to_many_q', 'text_change_many_q_to_not_q', 
    'text_change_many_q_to_single_q', 'text_change_not_q_to_many_q',
    'text_change_not_q_to_not_q', 'text_change_not_q_to_single_q', 
    'text_change_single_q_to_many_q', 'text_change_single_q_to_not_q', 
    'text_change_single_q_to_single_q'
]].sum(axis=1)

df_user_summaries2 = df_user_summaries[['id', 'score', 'mae_cursor_position', 
                   
                   'down_time_max', 'down_time_mean', 'down_time_std', 
                   'action_time_mean', 'action_time_std', 
                   'word_count_max', 'word_count_mean', 'word_count_std', 
                   
                   'activity_input', 'activity_movefrom', 'activity_nonproduction', 
                   'activity_paste', 'activity_removecut', 'activity_replace', 
                   
                   'down_event_alphanumeric', 'down_event_control_keys', 
                   'down_event_special_character', 'down_event_unknown', 
                   
                   'text_change_many_q', 'text_change_nochange', 'text_change_not_q', 
                   'text_change_single_q', 'text_change_replacements']]

df_user_summaries2

,id,score,mae_cursor_position,down_time_max,down_time_mean,down_time_std,action_time_mean,action_time_std,word_count_max,word_count_mean,...,activity_replace,down_event_alphanumeric,down_event_control_keys,down_event_special_character,down_event_unknown,text_change_many_q,text_change_nochange,text_change_not_q,text_change_single_q,text_change_replacements
0,001519c8,3.5,527.046930,1801877,8.481808e+05,395112.665961,116.246774,91.797374,256,128.116152,...,0.002738,0.633164,0.352366,0.014470,0.0,0.000782,0.046930,0.190849,0.758702,0.002738
1,0022f953,3.5,380.774654,1788842,5.188553e+05,384959.404177,112.221271,55.431189,323,182.714751,...,0.000407,0.607172,0.371231,0.021597,0.0,0.000000,0.103504,0.204156,0.691932,0.000407
2,0042269b,6.0,1238.755319,1771219,8.284918e+05,489500.796565,101.837766,82.383766,404,194.772727,...,0.001692,0.702128,0.286025,0.011847,0.0,0.000725,0.042311,0.167795,0.787476,0.001692
3,0059420b,2.0,152.393316,1404394,7.854830e+05,385205.014399,121.848329,113.768226,206,103.618895,...,0.000643,0.670951,0.315553,0.013496,0.0,0.000643,0.063625,0.198586,0.736504,0.000643
4,0075873a,4.0,640.161596,1662390,7.133542e+05,405576.409034,123.943896,62.082013,252,125.082971,...,0.000000,0.608850,0.364678,0.026472,0.0,0.000000,0.028447,0.195575,0.775978,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,3.5,1474.277696,1791581,7.361019e+05,503882.020411,105.437856,63.622575,461,256.353661,...,0.000422,0.600127,0.381304,0.018569,0.0,0.000211,0.039882,0.198776,0.760709,0.000422
2467,ffbef7e5,4.0,206.658986,1799124,8.419662e+05,512744.745940,82.266129,36.178818,438,223.013057,...,0.000384,0.719662,0.256144,0.024194,0.0,0.000000,0.056836,0.205453,0.737327,0.000384
2468,ffccd6fd,1.5,133.272935,1959273,1.229015e+06,514320.848199,75.605615,63.494975,201,157.589292,...,0.000000,0.316357,0.681358,0.002285,0.0,0.000000,0.041136,0.622266,0.336598,0.000000
2469,ffec5b38,5.0,398.298890,1508335,5.765185e+05,334477.976640,89.277915,54.515788,413,205.917027,...,0.000000,0.728254,0.249846,0.021900,0.0,0.000617,0.021900,0.177668,0.799815,0.000000


some data patterns to consider when reducing "text change" features to pooled features...

In [36]:
df_train_clean.loc[df_train_clean["text_change"].str.contains("=>"), :][["activity", "down_event", "text_change"]].value_counts()

activity  down_event         text_change         
replace   alphanumeric       many_q => single_q      3633
                             single_q => single_q     258
          control_keys       many_q => not_q          212
          special_character  many_q => not_q          132
          alphanumeric       many_q => many_q          51
                             not_q => single_q         36
          special_character  not_q => not_q            36
          control_keys       not_q => not_q            28
          special_character  single_q => not_q         18
          control_keys       many_q => many_q          16
                             single_q => not_q         14
                             single_q => single_q       6
          alphanumeric       single_q => many_q         3
          control_keys       not_q => many_q            3
          alphanumeric       many_q => not_q            1
                             not_q => many_q            1
dtype: int64

In [37]:
df_train_clean.loc[df_train_clean["text_change"].isin(["many_q", "single_q", "not_q"]), :][["activity", "down_event", "text_change"]].value_counts()

activity    down_event         text_change
input       alphanumeric       single_q       5361416
            control_keys       not_q          1229167
remove/cut  control_keys       single_q        766755
                               not_q           200133
input       special_character  not_q           136175
remove/cut  control_keys       many_q            3054
paste       alphanumeric       many_q             331
            control_keys       not_q              175
remove/cut  alphanumeric       many_q             138
                               single_q            61
paste       control_keys       many_q              56
movefrom    control_keys       many_q              35
paste       alphanumeric       single_q            34
input       unknown            not_q               32
remove/cut  alphanumeric       not_q               17
input       alphanumeric       not_q                6
movefrom    control_keys       single_q             4
            alphanumeric       many_q  

In [38]:
# save df_user_summaries2 to csv (a possible subset of features)
df_user_summaries2.to_csv(data_path + "df_user_summaries_select.csv", index=False)

In [39]:
# time to run whole file
duration = time.time() - start
print("Time taken to run whole file: {} minutes".format(round(duration/60, 2)))

Time taken to run whole file: 22.82 minutes
